In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyreadr
from scipy.linalg import toeplitz
import doubleml as dml
from sklearn.linear_model import LogisticRegression, LinearRegression
import sklearn
sklearn_version = sklearn.__version__


def dgp_atte(n_obs=500, dim_x=20, theta=0, R2_d=0.5, R2_y=0.5,schwellwert=0):
    """"
    Generiert Daten aus (IRM) model.
    The data generating process is inspired by a process used in the simulation experiment"""

    v = np.random.uniform(size=[n_obs, ])
    zeta = np.random.standard_normal(size=[n_obs, ])
    cov_mat = toeplitz([np.power(0.5, k) for k in range(dim_x)])
    x = np.random.multivariate_normal(np.zeros(dim_x), cov_mat, size=[n_obs, ])
    beta = [1 / (k**2) for k in range(1, dim_x + 1)]
    b_sigma_b = np.dot(np.dot(cov_mat, beta), beta)
    c_y = np.sqrt(R2_y/((1-R2_y) * b_sigma_b))
    c_d = np.sqrt(np.pi**2 / 3. * R2_d/((1-R2_d) * b_sigma_b))
    xx = np.exp(np.dot(x, np.multiply(beta, c_d)))
    d=np.zeros(n_obs)
    d = 1. * ((xx/(1+xx)) > v)





    #Um zu gewährleisten, dass viele Propensityscores nahe bei 0 oder 1 liegen wird hier ein % Wert festgelegt,
    #der in etwa dem Minimum %-Wert an Beobachungen entspricht, der nahe bei 0 oder 1 liegen soll

    # Berechnung des (Schwellwert/2) % Perzentils von X1, um 
    #d für alle X1 Werte, die in den top (schwellwert/2 %) 
    #sind auf 1 zu setzen
    perzentil_X1 = np.percentile(x[:, 0], 100-schwellwert/2)
    # Setzen von d auf 1, wenn X1 größer ist 
    #als das schwellwert/2. Perzentil
    for i in range(n_obs):
        if x[i, 0] > perzentil_X1:
            d[i] = 1

    # Berechnung des (Schwellwert/2) Perzentils von X2 
    #analog zu X1 Manipulation nur, dass d auf 0 gesetzt 
    #wird wenn X2 klein ist
    perzentil_X2 = np.percentile(x[:, 1], (schwellwert)/2)

    for i in range(n_obs):
        if x[i, 1] < perzentil_X2:
            d[i] = 0
        
    #print("perzx2:",perzentil_X2,"min:",min(x[:, 1]))

    #Weiterführung der ursprünglichen Datengenerierung
    y = d * theta + d * np.dot(x, np.multiply(beta, c_y)) + zeta
    y0 = zeta
    y1 = theta + np.dot(x, np.multiply(beta, c_y)) + zeta

    #Berechnung von ATTE und ATE
    ATTE = np.mean(y1[d == 1] - y0[d == 1])
    ATE = np.mean(y1 - y0)
    x_cols = [f'X{i + 1}' for i in np.arange(dim_x)]
    data = pd.DataFrame(np.column_stack((x, y, d)),
    columns=x_cols + ['y', 'd'])
    
    results = {'ATTE': ATTE,
    'ATE': ATE,
    'data': data}
    return results


if sklearn_version == '1.0.2':
    penalty = 'none'
else:
    penalty = None
    
# Use linear and logistic regression in DoubleML
ml_g = LinearRegression()
ml_m = LogisticRegression(penalty=penalty)

# Helper function to calculate whether CI covers true value
def cover_true(theta, ki):
    """
    Function to check whether theta is contained in confindence interval.
    Returns 1 if true and 0 otherwise.
    Input:
    ------
    theta: true value of target coefficent (ATE or ATTE ...)
    confint: pd.series with lower and upper confidence bound
    """
   
    

    if(ki.loc["d","2.5 %"] < theta and theta < ki.loc["d","97.5 %"]):
        return  1
    else:
    
        return 0

In [33]:
#Anzahl Simulationsdurchläufe
n_sim = 1000
#Anzahl der erzeugten Objekte je Simulationsdurchlauf
n_obs = 800
#Anzahl der zu erzeugenden Variablen exklusive D und Y
dim_x = 10
#Parameter Theta für den DGP festlegen
theta = 3
#Niveau des Konfidenzinterfalls bestimmen
level = 0.95
# Score art festlegen (ATTE oder ATE)
score = "ATTE"
#Schwellwert festlegen der ganz grob bestimmt, um wie stark der Anteil an ps-Werten nahe bei 0 oder 1  erhöht wird 
schwellwert = 50

#Datenobjekte zur Speicherung der Daten Ergebnisse der einzelnen Simulationsdurchläufe anlegen
sim_data = [None]*n_sim
sim_ergebnisse = [None]*n_sim


#Daten generieren und speichern für alle Durchläufe
np.random.seed(431)

for index_durchlauf in range(n_sim):
    meineDaten = dgp_atte(theta = theta, n_obs = n_obs, dim_x = dim_x,schwellwert=schwellwert)
    sim_data[index_durchlauf] = meineDaten

 
    


for index_durchlauf in range(n_sim):
    data_dieserDurchlauf = sim_data[index_durchlauf]  

    #Die eigentlichen Daten und den wahren ATE/ATTE extrahieren
    data_extract=data_dieserDurchlauf['data']
    truescore = data_dieserDurchlauf[score]
    
    #Daten in das DML Datenformat übergeben
    data_dml = dml.DoubleMLData(data_extract, 'y', 'd')

    #Für die 3 implementierten Varianten ein seperates dml_irm Objekt erzeugen mit den selben Daten, um Vergleichbarkeit zu gewährleisten
    #dml_irm_none = dml.DoubleMLIRM(data_dml, ml_g, ml_m, score = score, trimming_rule="none")
    dml_irm_discard = dml.DoubleMLIRM(data_dml, ml_g, ml_m, score = score, trimming_rule="discard")
    dml_irm_truncate = dml.DoubleMLIRM(data_dml, ml_g, ml_m, score = score, trimming_rule="truncate")

    #Modelle aufstellen
    #dml_irm_none.fit(store_predictions=True)
    dml_irm_discard.fit(store_predictions=True)
    dml_irm_truncate.fit(store_predictions=True)


    #ps aus den predictions extrahieren und formatieren
    #ps_none = pd.Series(dml_irm_none.predictions['ml_m'].reshape(dml_irm_none.predictions['ml_m'].size), name = 'ps_none')
    ps_discard = pd.Series(dml_irm_discard.predictions['ml_m'].reshape(dml_irm_discard.predictions['ml_m'].size), name = 'ps_discard')
    ps_truncate = pd.Series(dml_irm_truncate.predictions['ml_m'].reshape(dml_irm_truncate.predictions['ml_m'].size), name = 'ps_truncate')

    # ps und ursprüngliche Daten in einem Objekt zusammenfassen als Vorbereitung für die Visualisierung
    #newdata_none= pd.concat([data_extract,ps_none], axis=1)
    newdata_discard= pd.concat([data_extract,ps_discard], axis=1)
    newdata_truncate= pd.concat([data_extract,ps_truncate], axis=1)

    #Koeffizienten speichern
    #coef_none = dml_irm_none.coef
    coef_discard = dml_irm_discard.coef
    coef_truncate = dml_irm_truncate.coef

    #Standartfehler speichern
    #se_none = dml_irm_none.se
    se_discard = dml_irm_discard.se
    se_truncate = dml_irm_truncate.se

    #Konfidenzintervalle speichern
    #ki_none = dml_irm_none.confint(level = level)
    ki_discard =  dml_irm_discard.confint(level = level)
    ki_truncate =  dml_irm_truncate.confint(level = level)

    #Spannweite der KI speichern
    #ki_none_spannweite = ki_none.loc["d","97.5 %"] - ki_none.loc["d","2.5 %"]  
    ki_discard_spannweite = ki_discard.loc["d","97.5 %"] - ki_discard.loc["d","2.5 %"]  
    ki_truncate_spannweite = ki_truncate.loc["d","97.5 %"] -  ki_truncate.loc["d","2.5 %"]  

  

    #prüfen und speichern, ob das Konfidenzinterfall des Schätzers den wahren Wert umfasst
    #kihit_none = cover_true(truescore,ki_none)
    kihit_discard = cover_true(truescore,ki_discard)
    kihit_truncate = cover_true(truescore,ki_truncate)

  


    #Bias berechnen
    #Bias_none = truescore - coef_none
    Bias_discard =  truescore - coef_discard 
    Bias_truncate= truescore - coef_truncate 

    #absoluten Bias berechnen
    #aBias_none = np.abs(Bias_none)
    aBias_discard =  np.abs(Bias_discard) 
    aBias_truncate= np.abs(Bias_truncate)

    #quadrierten Bias berechnen
    #qBias_none = np.square(Bias_none)
    qBias_discard =  np.square(Bias_discard) 
    qBias_truncate= np.square(Bias_truncate)

    #standartisierten Bias berechnen
    #sBias_none = aBias_none / dml_irm_none.se
    sBias_discard = aBias_discard / dml_irm_discard.se
    sBias_truncate = aBias_truncate / dml_irm_truncate.se

    # Berechnen des prozentualen Anteils der Beobachtungen, bei denen ps größer als 0,99 oder kleiner als 0,1 ist und somit nan
    #
    anteil = ps_discard.isna().mean()
    
    
    #Deskriptive Statistiken erzeugen
    #ds_none=ps_none.describe(percentiles=[.01, .05, .10, .25, .50, .75, .90, .95, .99]) 
    ds_discard=ps_discard.describe(percentiles=[.01, .05, .10, .25, .50, .75, .90, .95, .99]) 
    ds_truncate=ps_truncate.describe(percentiles=[.01, .05, .10, .25, .50, .75, .90, .95, .99]) 
    #print("ds_none:",ds_none,type(ds_none))


    ergebnisse = pd.DataFrame.from_dict({"Anteil":[anteil],"ATE/ATTE":[truescore],
                                            "Dcoef":[coef_discard],"Tcoef":[coef_truncate],
                                            "Dse":[se_discard],"Tse":[se_truncate],
                                            "Dbias":[Bias_discard],"Tbias":[Bias_truncate],
                                            "DaBias":[aBias_discard],  "TaBias":[aBias_truncate],
                                            "DqBias":[qBias_discard], "TqBias":[qBias_truncate],
                                            "DsBias":[sBias_discard],   "TsBias":[sBias_truncate],
                                            "DinKI":[kihit_discard],"TinKI":[kihit_truncate],
                                            "SKI_D":[ki_discard_spannweite],"SKI_T":[ki_truncate_spannweite],
                                            "n_obs":[n_obs] })


    
    sim_ergebnisse[index_durchlauf] = ergebnisse
    #

    # "Ncoef":[coef_none], "Nse":[se_none], "Nbias":[Bias_none], "NaBias":[aBias_none], "NqBias":[qBias_none], "NsBias":[sBias_none], "NinKI":[kihit_none],
    #"SKI_N":[ki_none_spannweite],
    
    


    #CODE AB HIER AUSKOMMENTIERT LASSEN ODER ANZAHL SIMULATIONEN (n_sim)  AUF 1 SETZEN!!!!!!
    #Diente den Voruntersuchungen und erzeugt für JEDEN Durchlauf 3 Histogramme und 3 Boxplots für die Verteilung des ps.
    #Wurde an dieser Stelle bewusst behalten, weil praktisch, komfortabel und ganz spannend wenn man ein wenig rumspielen/untersuchen möchte


    

    

# Boxplot für Variante none    
#plt.figure(figsize=(5,3))
#plt.boxplot(ps_none,vert=False)
#plt.title(dml_irm_none.trimming_rule)
#plt.xlabel("ps")
#plt.show()

# # #Der Boxplot kann mit den nan werten nicht umgehen, die beim discarden erzeugt werden:
# ps_discard=ps_discard.dropna(axis=0)

# # Boxplot für Variante discard
# plt.figure(figsize=(5,3))
# plt.boxplot(ps_discard,vert=False)
# plt.title(dml_irm_discard.trimming_rule)
# plt.xlabel("ps")
# plt.show()

# # Boxplot für Variante truncate    
# plt.figure(figsize=(5,3))
# plt.boxplot(ps_truncate,vert=False)
# plt.title(dml_irm_truncate.trimming_rule)
# plt.xlabel("ps")
# plt.show()

# #Aufteilen nach treatmentstatus, um die Verteilung des ps in Abhängigkeit vom Treatmentstatus darzustellen
# #d1_data_none = newdata_none[newdata_none['d'] == 1]
# #d0_data_none = newdata_none[newdata_none['d'] == 0]

# d1_data_discard = newdata_discard[newdata_discard['d'] == 1]
# d0_data_discard = newdata_discard[newdata_discard['d'] == 0]

# d1_data_truncate = newdata_truncate[newdata_truncate['d'] == 1]
# d0_data_truncate = newdata_truncate[newdata_truncate['d'] == 0]

# #Erstellen des Histogramms für Verteilung des (ps | d) Variante none
# #plt.hist(d1_data_none['ps_none'], bins=20, alpha=0.8, label='d = 1')
# #plt.hist(d0_data_none['ps_none'], bins=20, alpha=0.5, label='d = 0')
# #plt.xlabel('ps')
# #plt.ylabel('Absolute Anzahl')
# #plt.title(dml_irm_none.trimming_rule)
# #plt.show()

# #Erstellen des Histogramms für Verteilung des (ps | d)  Variante discard
# plt.hist(d1_data_discard['ps_discard'], bins=20, alpha=0.8, label='d = 1')
# plt.hist(d0_data_discard['ps_discard'], bins=20, alpha=0.5, label='d = 0')
# plt.xlabel('ps')
# plt.ylabel('Absolute Anzahl')
# plt.title(dml_irm_discard.trimming_rule)
# plt.show()

# # #Erstellen des Histogramms  für Verteilung des (ps | d)  Variante truncate
# plt.hist(d1_data_truncate['ps_truncate'], bins=20, alpha=0.8, label='d = 1')
# plt.hist(d0_data_truncate['ps_truncate'], bins=20, alpha=0.5, label='d = 0')
# plt.xlabel('ps')
# plt.ylabel('Absolute Anzahl')
# plt.title(dml_irm_truncate.trimming_rule)
# plt.show()

    
#Ergebnisse speichern
speicher = pd.concat(sim_ergebnisse)

#Ergebnisse zusammenfassen und ausgeben
zusammenfassung = speicher.mean(axis=0)
print("zusammenfassung:","\n",zusammenfassung)

#RMSE berechnen:
#rmse_none = np.sqrt(zusammenfassung["NqBias"])
rmse_discard = np.sqrt(zusammenfassung["DqBias"])
rmse_truncate = np.sqrt(zusammenfassung["TqBias"])   
print("rmse_discard:",rmse_discard,"\n","rmse_truncate:",rmse_truncate)            
    

zusammenfassung: 
 Anteil      0.117991
ATE/ATTE    3.707617
Dcoef       3.621245
Tcoef       3.706148
Dse         0.191325
Tse         0.180189
Dbias       0.086372
Tbias       0.001469
DaBias       0.17774
TaBias      0.155519
DqBias      0.053859
TqBias      0.042119
DsBias      0.959358
TsBias      0.874314
DinKI          0.904
TinKI          0.936
SKI_D       0.749982
SKI_T       0.706328
n_obs          800.0
dtype: object
rmse_discard: 0.23207571744467745 
 rmse_truncate: 0.20522977875634954


In [60]:
def discard_naive(dataparameter,rule):

    #Übergabe der Daten und Aufstellen des Modells
    data_dml = dml.DoubleMLData(dataparameter,
                                 'y', 'd')
    dml_irm = dml.DoubleMLIRM(data_dml, ml_g, ml_m,
                               score = 'ATTE',
                                 trimming_rule=rule)
    dml_irm.fit()

    #ps aus den predictions extrahieren und 
    #formatieren, um an newdata anzuhängen
    ps = pd.Series(dml_irm.predictions['ml_m'].
                   reshape(dml_irm.predictions['ml_m'].
                           size), name = 'ps')
    newdata= pd.concat([dataparameter,ps], axis=1)

    print('datenobjektgröße prä:', newdata.shape)
    print('maxPS:' , max(newdata['ps']))
    print('minPS:' , min(newdata['ps']))


    

    #Aufteilen nach treatmentstatus, um ursprüngliche Daten darzustellen
    d1_data = newdata[newdata['d'] == 1]
    d0_data = newdata[newdata['d'] == 0]

    #2 Erstellen des Histogramms aller Daten für Verteilung des ps
    # plt.hist(d1_data['ps'], bins=20, alpha=0.8, label='d = 1')
    # plt.hist(d0_data['ps'], bins=20, alpha=0.5, label='d = 0')
    # plt.xlabel('ps')
    # plt.ylabel('Absolute Anzahl')
    # plt.legend('a')
    # plt.show()

    #Observierungen mit ps kleiner trimmingthreshold
    #oder  größer 1-trimmmingthreshold löschen
  
    
    newdata_droped=newdata.drop(newdata[(newdata['ps']<=dml_irm.trimming_threshold)|(newdata['ps']>=(1-dml_irm.trimming_threshold))].index)
    newdata_droped=newdata_droped.reset_index(drop=True)
    #print('newdata_droped:' ,type(newdata_droped), newdata_droped.shape, newdata_droped.size)
    #print('maxd:' , max(newdata_droped['ps']))
    #print('mind:' , min(newdata_droped['ps']))

    #erneut aufteilen um nun ps verteilung des bereinigten Datensatzes darstellen
    d1_data_droped = newdata_droped[newdata_droped['d'] == 1]
    d0_data_droped = newdata_droped[newdata_droped['d'] == 0]

    # Erstellen des Histogramms für bereinigte Daten
    # plt.hist(d1_data_droped['ps'], bins=20, alpha=0.8, label='d = 1')
    # plt.hist(d0_data_droped['ps'], bins=20, alpha=0.5, label='d = 0')
    # plt.xlabel('ps')
    # plt.ylabel('Absolute Anzahl')
    # plt.legend('b')
    # plt.show()

    print('datenobjektgröße post:' ,newdata_droped.shape)
    print('maxPS:' , max(newdata_droped['ps']))
    print('minPS:' , min(newdata_droped['ps']))
    
    #ps wird nun wieder entfernt, da nur mit den 
    #bereinigten Daten der gesamte Vorgang 
    #wiederholt werden soll.
    newdata = newdata_droped.drop('ps', axis=1)

    
    
    return newdata



meineDaten = dgp_atte(theta = 3,
                       n_obs = 200,
                         dim_x = 10,
                         schwellwert=0)
nurDaten=meineDaten['data']


np.random.seed(4321)

a = discard_naive(dataparameter=nurDaten,
                  rule = "none")


print("\n","\n",'Durchlauf 2:', "\n")
b = discard_naive(a,rule="none")

print("\n","\n",'Durchlauf 3:', '\n')
c = discard_naive(b,rule="none")

print("\n","\n",'Durchlauf 4:', '\n')
d = discard_naive(c,rule="none")

print("\n","\n",'Durchlauf 5:', '\n')
e = discard_naive(d,rule="none")

print("\n","\n",'Durchlauf 6:', "\n")
f = discard_naive(e,rule="none")

print("\n","\n",'Durchlauf 7:', '\n')
g = discard_naive(f,rule="none")

print("\n","\n",'Durchlauf 8:', '\n')
h = discard_naive(g,rule="none")

print("\n","\n",'Durchlauf 9:', '\n')
i = discard_naive(h,rule="none")

datenobjektgröße prä: (200, 13)
maxPS: 0.9995713217488895
minPS: 0.00040052568799330685
datenobjektgröße post: (181, 13)
maxPS: 0.989922806408379
minPS: 0.011609179148799813

 
 Durchlauf 2: 

datenobjektgröße prä: (181, 13)
maxPS: 0.9993532714406215
minPS: 0.0034465159443917794
datenobjektgröße post: (171, 13)
maxPS: 0.9877666895515048
minPS: 0.010947725253238992

 
 Durchlauf 3: 

datenobjektgröße prä: (171, 13)
maxPS: 0.9987720232006024
minPS: 0.008437245172404266
datenobjektgröße post: (169, 13)
maxPS: 0.988157713067804
minPS: 0.012645028329871383

 
 Durchlauf 4: 

datenobjektgröße prä: (169, 13)
maxPS: 0.9893909734021283
minPS: 0.010258503312758205
datenobjektgröße post: (169, 13)
maxPS: 0.9893909734021283
minPS: 0.010258503312758205

 
 Durchlauf 5: 

datenobjektgröße prä: (169, 13)
maxPS: 0.9911485671038283
minPS: 0.005894849475390491
datenobjektgröße post: (165, 13)
maxPS: 0.988464333415763
minPS: 0.013405840997927744

 
 Durchlauf 6: 

datenobjektgröße prä: (165, 13)
maxPS: 0